In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy           # Necessário para copiar os dados dentro de funções e evitar alterações inplace dos dados
                                    # Isso para que as funções recebam um dado e gerem um dado novo, mantendo o original inalterado.
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm

from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

In [2]:
from preprocessor import Preprocessor

In [4]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, Perceptron, LinearRegression

In [6]:
# Importação dos dados
DATA_PATH = 'dados/dados_treino.csv'
prep = Preprocessor(DATA_PATH)

# Processamento e adequação do dataset
prep.preprocess_dataset()

# Escolha do usuário para treinamento / análise
USER = '#ID9181'
user_data = prep.get_user_data(USER)

# Separação do dataset de treino e teste
X_train, X_test, y_train, y_test = prep.train_test_split(user_data, oversampling="SMOTENC")

#  <<<<<< Aplicar métodos de aprendizado de máquina >>>>>>>

# clf = svm.SVC(kernel='linear') # Linear Kernel
clf = Perceptron()
# clf.fit(X_train, y_train) # Sem SMOTE
clf.fit(X_train, y_train)  # Com SMOTE

y_pred = clf.predict(X_test)

from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))


Usuarios: ['#ID4940', '#ID9181']
Labels shape: (2403,)
Features shape(2403, 43)
with SMOTENC oversampling
Accuracy: 0.14137214137214138
Precision: 0.14137214137214138
Recall: 1.0
